In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 18 17:56:29 2022

@author: Emanuel Mendes
"""

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint
from scipy.optimize import fsolve


#diametro fuselagem = 1.55 m

def eta(P, V):
    eta = fsolve(lambda n: n - 2/(np.sqrt((P*n/V)/((V**2)*np.pi*((1.25/2)**2)*1.038/2) + 1) + 1),1)
    return eta[0]

def taper_function(lbd):
    f = 0.0524*lbd**4 - 0.15*lbd**3 + 0.1659*lbd**2 -0.0706*lbd + 0.0119
    return f

def taper_ratio(AR, b, Cr):
    lambd = (2*b/(AR*Cr)) - 1
    return lambd

def oswald_number(AR, Cr, df, b, Cd0):
    #Kroo-based method
    #Proposed by M. Nita, D. Scholz
    
    #Zero sweep, trapezoidal wing
    e_theo = 1/(1 + taper_function(taper_ratio(AR,b,Cr) - 0.093)*AR)
    
    #Velocidade do som ~350 kph, velocidade maxima na tabela comparativa ~190 kph
    #Regime incompressível, logo:
    k_em = 1
    
    k_ef = 1 - 2*(df/b)**2
    
    Q = 1/(e_theo*k_ef)
    P = 0.38*Cd0
    e = k_em/(Q + P*np.pi*AR)
    return e

def Cf(Re,laminar):
    if laminar:
        Cf = 1.328/(Re**0.5)
    else:
        Cf = 0.455/(np.log10(Re)**2.58)
    return Cf

def CD0(V,rho,b,AR,df,tc):
    d = b/AR
    Re = rho*V*d/(1.918E-5)
    length = 3.5
    
    #Surface roughness, characteristic length
    #smooth paint = 0.0634
    #sheet metal = 0.0405
    #polished metal = 0.0152
    #Smooth composite = 0.0052
    k = 3E-6 #chute educado -> roughness
    Re_cutoff = 38.21*(0.0052/k)**1.053
    Re = min(Re,Re_cutoff)
    cf = Cf(Re,False)
    
    S_ref_wing = (b**2)/AR
    S_ref_fus = np.pi*df*length
    S_ref = S_ref_wing + S_ref_fus
    S_wet_wing = S_ref_wing*(1.977 + 0.52*tc)
    e = np.sqrt(1 - (df**2)/(length**2))
    S_wet_fus = 2*(np.pi*df**2)*(1 + (length/df)*np.arcsin(e)/e)
    S_wet = S_wet_wing + S_wet_fus
    
    cd0 = cf*(S_wet/S_ref)
    return cd0

def W_motor(P):
    P_unit = P/2
    W = P_unit/4500  #5000W/kg for an EMRAX
    if (W<=7.2):
        return 7.2*2
    else:
        if (W<=9.3):
            return 9.3*2
        else:
            if(W<=12.3):
                return 12.3*2
            else:
                if (W<=20.3):
                    return 20.3*2
                else:
                    if (W<=41.5):
                        return 41.5*2
                    else:
                        return W*2

def W_empty(AR):
    w = 22.2*AR
    if(w<180):
        return 180
    else:
        return w

def CD(V,rho,b,AR,df,tc, Cl):
    Cd0 = CD0(V,rho, b, AR, df, tc)
    Cr = (2*b/(AR))/1.45
    e = oswald_number(AR,Cr,df,b,Cd0)
    Cd = Cd0 + (Cl**2)/(np.pi*AR*e)
    return Cd

def Restricoes(x):
    #Atribuindo variáveis independentes
    P = x[0]
    W_batt = x[1]
    W_load = x[2]
    AR = x[3]
    b = x[4]
    Cl = x[5]
    V = x[6]
    Cl_climb = x[7]
    V_climb = x[8]
    W = 600*9.81
    rho = 1.038 #ISA @3000m Altitude +20°C
    rho_climb = 1.111 #@ISA @1000ft Altitude +20°C
    df = 1.55
    tc = 0.1
        
    P_max = W_motor(P)*9000 #9000W/kg peak 5000W/kg sustained
    S = (1/AR)*(b**2)
    
    Cd_climb = CD(V_climb,rho_climb, b, AR, df, tc, Cl_climb)
    h_dot = (P_max*0.92*eta(P_max/2,V))/W - (((1/2)*S*rho_climb*Cd_climb*(V_climb**2))/W)*V_climb
    
    Cd = CD(V,rho, b, AR, df, tc, Cl)
    Vc = (2*P*eta(P/2,V)*0.92/(rho*S*Cd))**(1/3)  #eficiencia propulsiva = 0.88
    #Atribuindo restrições
    y1 = h_dot
    y2 = h_dot - Vc
    y3 = Vc - V
    y4 = (1/2)*rho*(Vc**2)*S*Cl - W
    y5 = ((b**2)/S) - AR
    y6 = W_motor(P) + W_load + W_batt + W_empty(AR) - (W/9.81)
    y7 = b
    y8 = eta(P/2,Vc)*P*0.92/Vc - (1/2)*rho*(V**2)*S*Cd
    return ([y1,y2,y3,y4,y5,y6,y7,y8])

#lower boundary
lb = [20, -200, 0, 0, 0, 0, 1, 0]

#upper boundary
ub = [200, 0, 0, 0, 0, 0, 11, 0]

#initial guess
x0 = [82000, 200, 232.8, 8, 11, 0.25, 55, 0.35, 35]



##################################
# objective function
def obj_fun(x):
    #Atribuindo variáveis independentes
    P = x[0]
    W_batt = x[1]
    W_load = x[2]
    AR = x[3]
    b = x[4]
    Cl = x[5]
    V = x[6]
    Cl_climb = x[7]
    V_climb = x[8]
    W = 600*9.81
    rho = 1.038 #ISA @3000m Altitude +20°C
    rho_climb = 1.111 #@ISA @1000ft Altitude +20°C
    df = 1.55
    tc = 0.1
    
    Cd = CD(V,rho, b, AR, df, tc, Cl)
    S = (1/AR)*(b**2)
    Vc = (2*P*0.92*eta(P/2,V)/(rho*S*Cd))**(1/3)
    
    Range = ((W_batt*200*3600*0.8)/P)*(V)/1000
    Payload = W_load
    objt = -1*Range*Payload*(V*3.6)
    
    return objt


#Bounds
bnds = ((0, 415000), (0, 600), (0,600), (1,None), (1, 11), (0.1, 1), (23,None), (0.2,1), (23,None))


#Declarando restrição
constr = NonlinearConstraint(Restricoes, lb, ub)


# Result
result = minimize(obj_fun, x0, method = 'trust-constr', constraints = constr, bounds = bnds, options= {'maxiter': 500000})
print(result)

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 440194
      cg_stop_cond: 2
            constr: [array([ 2.27444907e+01, -5.54988616e+01,  1.42108547e-14,  2.72848411e-12,
        0.00000000e+00,  0.00000000e+00,  1.07255440e+01,  1.36424205e-12]), array([8.19999526e+04, 1.86764631e+02, 2.14635369e+02, 8.07063151e+00,
       1.07255440e+01, 1.29965135e-01, 7.82433523e+01, 5.20441678e-01,
       3.50323417e+01])]
       constr_nfev: [4177860, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 6.142637302029232e+16
  constr_violation: 2.7284841053187847e-12
    execution_time: 4264.899938344955
               fun: -6205863.298279414
              grad: array([ 7.56813025e+01, -3.32282574e+04, -2.89135164e+04,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.58629792e+05,  0.00000000e+00,
        0.00000000e+00])
               jac: [array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
   

In [ ]:
print(eta(50,50))

In [5]:
print(CD0(78.24,1.038, 10.72, 8.07, 1.55, 0.1))

0.019807146933338897


In [7]:
print(CD(35,1.111, 10.72, 8.07, 0.1, 1.523, 0.52044))

0.03299503382806471


In [6]:
print(CD(78.24, 1.038, 10.72, 8.07, 0.1, 1.523, 0.1299))

0.020886217206137942


In [ ]:
print((10.35**2)/(8.042))

In [8]:
print(oswald_number(8.07, 1.825, 1.55, 10.72, 0.0198))

0.7999239037919973
